In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier

In [2]:
train_data = pd.read_csv('data_creation/training_data.csv', header=None)
train_X = train_data[train_data.columns[:-1]]
train_y = train_data[train_data.columns[-1]]
test_data = pd.read_csv('data_creation/test_data.csv', header=None)
test_X = test_data[test_data.columns[:-1]]
test_y = test_data[test_data.columns[-1]]
#print(f"Training dataset size: {len(train_X.index)}")
#print(f"Test dataset size: {len(test_X.index)}")
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,1660.000000,12.000000,5.333333,5.666667,1.000000,12.333333,0.692857,0.0,13.055783,26.258952,...,48.765331,63.902279,0.326692,0.087500,0.445602,0.0,0.974359,0.025641,44.428330,0
1,1660.000000,12.000000,5.333333,5.666667,1.000000,12.333333,0.692857,0.0,13.055783,26.258952,...,48.765331,63.902279,0.326692,0.087500,0.445602,0.0,0.974359,0.025641,44.428330,0
2,1660.000000,12.000000,5.333333,5.666667,1.000000,12.333333,0.692857,0.0,13.055783,26.258952,...,31.103462,45.165862,0.621176,0.014493,0.009259,0.0,0.838969,0.100242,33.628019,0
3,1660.000000,12.000000,5.333333,5.666667,1.000000,12.333333,0.692857,0.0,13.055783,26.258952,...,31.103462,45.165862,0.621176,0.014493,0.009259,0.0,0.838969,0.100242,33.628019,0
4,1549.666667,2.333333,0.666667,1.000000,0.666667,6.666667,0.259259,0.0,12.092857,18.333333,...,35.036562,48.044634,0.556505,0.063628,0.189934,0.0,0.840931,0.121083,33.346154,1


In [3]:
# converting to lists
X_train = []
y_train = []
X_test = []
y_test = []

for X, y, data in ((X_train, y_train, train_data), (X_test, y_test, test_data)):
    for idx in data.index:
        row = list(data.iloc[idx])
        X.append(row[:-1])
        y.append(row[-1])

In [4]:
print(len(X_train), len(X_train[0]))
print(len(y_train))

print(len(X_test), len(X_test[0]))
print(len(y_test))

16434 58
16434
1588 58
1588


## Random Forest

In [5]:
random_forest = RandomForestClassifier(n_estimators=250)
random_forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [6]:
print(f"Acuracy: {random_forest.score(X_test, y_test)}")

Acuracy: 0.7562972292191436


## NN

In [17]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(58,)))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

opt = tf.keras.optimizers.SGD(momentum = 0.3, learning_rate=0.1)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               30208     
_________________________________________________________________
dense_15 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 257       
Total params: 161,793
Trainable params: 161,793
Non-trainable params: 0
_________________________________________________________________


In [7]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(58, activation=tf.nn.relu, input_shape=(58,)))
model.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

opt = tf.keras.optimizers.SGD(momentum = 0.3, learning_rate=0.1)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 58)                3422      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1888      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 8,511
Trainable params: 8,511
Non-trainable params: 0
______________________________________________________

In [8]:
NUM_EPOCHS = 50
model.fit(X_train, y_train, epochs=NUM_EPOCHS, validation_data=(X_test, y_test))

Train on 16434 samples, validate on 1588 samples
Epoch 1/50
16434/16434 [==============================] - 5s 303us/sample - loss: 0.8313 - accuracy: 0.7212 - val_loss: 0.6593 - val_accuracy: 0.7059
Epoch 2/50
16434/16434 [==============================] - 3s 179us/sample - loss: 0.6077 - accuracy: 0.7267 - val_loss: 0.4464 - val_accuracy: 0.7733
Epoch 3/50
16434/16434 [==============================] - 3s 202us/sample - loss: 0.5537 - accuracy: 0.7388 - val_loss: 0.6069 - val_accuracy: 0.7084
Epoch 4/50
16434/16434 [==============================] - 4s 215us/sample - loss: 0.5235 - accuracy: 0.7463 - val_loss: 0.4342 - val_accuracy: 0.7827
Epoch 5/50
16434/16434 [==============================] - 4s 217us/sample - loss: 0.5080 - accuracy: 0.7503 - val_loss: 0.4360 - val_accuracy: 0.7815
Epoch 6/50
16434/16434 [==============================] - 4s 228us/sample - loss: 0.4981 - accuracy: 0.7555 - val_loss: 0.4467 - val_accuracy: 0.7783
Epoch 7/50
16434/16434 [===========================